<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/notebooks/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 935.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c0f1300d36a16d87a929fe6c1bfea692f03a79c969c5bf38fe46316bc3aebda4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read the data
nutrition - calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) ,carbohydrates (PDV)

In [68]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/RAW_recipes.csv')
df = df.reset_index(drop=True)
df.head()

CPU times: user 4.57 s, sys: 389 ms, total: 4.96 s
Wall time: 5.29 s


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Transform the data

In [70]:
## Split the nutrition column to separate columns
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df.nutrition.str.split(",",expand=True)

## Replace [] brackets after split
df['calories'] =  df['calories'].apply(lambda x: x.replace('[',''))
df['carbohydrates (PDV)'] =  df['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

## Convert the datatypes from str to float
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

## Drop unnecessary columns
df.drop(['id', 'contributor_id','submitted','tags','nutrition'], axis=1,inplace = True)

In [36]:
df.dtypes

name                    object
minutes                  int64
n_steps                  int64
steps                   object
description             object
ingredients             object
n_ingredients            int64
calories               float64
total fat (PDV)        float64
sugar (PDV)            float64
sodium (PDV)           float64
protein (PDV)          float64
saturated fat (PDV)    float64
carbohydrates (PDV)    float64
dtype: object

In [37]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Cleaning the ingredients column

In [72]:
%%time
## First convert the string representation of list to list
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","") #.str.split(",")

CPU times: user 151 ms, sys: 2 ms, total: 153 ms
Wall time: 153 ms


In [71]:
df['ingredients'].iloc[0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

Explode the series containing list to a single list:

In [10]:
ingredients = df['ingredients'].explode().str.lower().str.strip()
print(f"Around {ingredients.shape[0]} ingredients are there in the dataset")

Around 231637 ingredients are there in the dataset


Now find the unique ingredients:

In [11]:
unique_ingredients = ingredients.unique()
print(f"There around {unique_ingredients.shape[0]} are unique out of {ingredients.shape[0]}")

There around 230475 are unique out of 231637


There might be some same words that mean the same but with different name.

Try with some keywords to see if it exists like search for 'egg' or 'milk'

In [12]:
keyword = 'bread'
unique_word = ingredients[ingredients.str.contains(keyword.lower())].unique()
if len(unique_word)> 0:
    sample = ", ".join(np.random.choice(unique_word,10))
    print(f"There are around {len(unique_word)} unique ingredients containing the work 'f{keyword}'.\n\n Example:\n\n {sample}....")
else:
    print(f"No ingredients for the keyword: {keyword}")

There are around 18673 unique ingredients containing the work 'fbread'.

 Example:

 elbow macaroni, cabbage, onion, corned beef, cream of mushroom soup, milk, pimientos, mustard, pepper, dry breadcrumbs, butter, cheddar cheese, chicken, ham, swiss cheese, onion, butter, flour, milk, dry mustard, nutmeg, salt, pepper, breadcrumbs, cheddar cheese, bread, egg, peanut butter, sugar, butter, ground chicken, colby-monterey jack cheese, oscar mayer real bacon bits, chicken coating mix, barbecue sauce, ground beef, part-skim mozzarella cheese, breadcrumbs, ground pork, apple, coating mix, ground turkey, stove top stuffing mix, water, hickory flavored barbecue sauce, butter, fennel, leeks, milk, double cream, garlic cloves, tarragon, nutmeg, fennel seed, breadcrumbs, parmesan cheese, italian bread, olive oil, brown sugar, granny smith apples, pear, dried cherries, ground cinnamon, eggs, milk, almond extract, amaretto, toasted almond, olive oil, butter, onions, salt and pepper, fresh thyme, bay

## Sentence encoder model

In [13]:
%%time
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

CPU times: user 2.2 s, sys: 1.2 s, total: 3.4 s
Wall time: 32.3 s


In [14]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Encoding the ingredients

In [53]:
%%time

## encode all the ingredients:
embedding = model.encode(df['ingredients'].values)

CPU times: user 9min 13s, sys: 2.8 s, total: 9min 16s
Wall time: 9min 7s


In [55]:
embedding.shape

(231637, 768)

In [62]:
## Data frame to store the embeddings
e_cols = [f"e{e}" for e in range(embedding.shape[1])] ## Set the columns names for each word embedding
df_w = pd.DataFrame(embedding, columns = e_cols)
df_w.head()

,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e758,e759,e760,e761,e762,e763,e764,e765,e766,e767
0,-0.013716,-0.018651,-0.012355,-0.013950,-0.006719,0.040729,-0.102842,0.067343,0.034058,-0.007671,...,-0.039862,-0.039966,-0.025543,-0.033836,0.037627,0.046141,-0.023707,0.011108,-0.055300,-0.035544
1,-0.020377,-0.004778,0.018426,-0.034890,-0.014364,0.041468,-0.070624,0.073116,-0.031720,-0.005652,...,-0.007511,-0.051415,0.021640,-0.036801,0.027939,-0.003634,-0.004023,-0.022614,-0.054280,-0.001435
2,-0.008770,0.064799,-0.011579,-0.030650,-0.014053,0.053012,-0.082036,0.084343,-0.006779,-0.024982,...,0.001998,-0.027257,0.012747,-0.015481,0.029199,0.037459,-0.028463,0.005323,-0.044123,-0.027282
3,-0.004377,0.017181,-0.008179,-0.051775,-0.035114,0.057691,-0.035182,0.112658,0.017557,0.001312,...,-0.029760,-0.043967,0.022254,-0.032497,-0.008657,-0.016600,-0.026343,-0.001292,-0.019408,-0.031982
4,0.045295,0.028997,-0.014462,-0.060430,-0.027733,0.048291,-0.045620,0.090624,0.039336,-0.062209,...,-0.044573,-0.028139,0.015719,-0.027388,0.008950,-0.003074,0.001409,0.004274,-0.073295,-0.055511


In [73]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [75]:
df_final = df.join(df_w)
df_final.sample(2)

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),...,e758,e759,e760,e761,e762,e763,e764,e765,e766,e767
36566,carob zucchini nut bread vegan,75,6,"['preheat oven to 35o f', 'in a large mixing b...",this was put together as a combination of seve...,"canola oil, white sugar, brown sugar, vanilla ...",17,355.0,28.0,105.0,...,0.016545,-0.028795,-0.021265,-0.038667,0.040433,0.001163,-0.034400,0.019494,-0.051518,0.006406
159048,petty s chill and grill chicken marinade,980,5,"['mix ingredients well', 'poke holes into chic...",unusual but good: from sue bee honey ad in mag...,"honey, italian salad dressing, italian salad d...",7,452.0,35.0,185.0,...,-0.070556,-0.040874,0.069240,-0.039978,0.001253,-0.003404,0.010451,-0.035719,-0.035970,-0.025674


In [42]:
y = df.sample(10)
y.head(2)

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
60967,cranberry vinaigrette,10,5,"['put all ingredients , except the oil in a bl...",very tasty on romaine.,"jellied cranberry sauce, vinegar, sugar, carda...",5,1436.7,168.0,470.0,3.0,1.0,70.0,40.0
74224,e z seasoned bread crumbs,25,4,['take 12 slices of stale bread and let air dr...,if you are using this for breading such as sal...,"plain breadcrumbs, parsley flakes, garlic powd...",8,145.5,2.0,10.0,19.0,9.0,2.0,8.0


(1, 768, 10)

In [50]:
pd.Dataframe(x)

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [21]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    a = np.array(vec1).ravel()
    b = np.array(vec2).ravel()
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(a,b) / denom * 100, 2)

In [28]:
recipe = df.iloc[0]['name']
index = df.loc[df['name']==recipe, "ingredients"].index
if len(index)>0 :
    ingredients = df.iloc[index[0]]['ingredients']
    # df_sample['ingredients'].apply(lambda x: cosine_similarity())

index, ingredients

(Int64Index([0], dtype='int64'),
 'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt')

In [ ]:
%%time
vec1 = embedding[index]
def cs(x):
    return cosine_similarity(vec1,x)
sim = sorted(list(map(cs, embedding)), reverse=True)

CPU times: user 43.3 ms, sys: 983 µs, total: 44.3 ms
Wall time: 43.8 ms
